### Get a GeoJSON with Crop types, style and prepare it

In [1]:
import os
import geopandas as gp

from pathlib import Path

In [25]:
BASE = f"/home/{os.getenv('NB_USER')}/work"

WORKDIR = os.path.join(BASE, "notebooks/pw")
RESULTSDIR = os.path.join(BASE, "results/pw")

In [3]:
predictions_path = os.getenv("PREDICTIONS", os.path.join(WORKDIR, "clu_predictions_2020_reformatted.geojson")) 

if not predictions_path:
    raise RuntimeError("Add PREDICTIONS env var for calculations") 
predictions_path

'/home/jovyan/work/notebooks/pw/clu_predictions_2020_reformatted.geojson'

In [12]:
def read_geojson(path, crs="epsg:4326"):
    df = gp.read_file(predictions_path)
    if str(df.crs) != crs:
        df.to_crs(crs, inplace=True)
    return df

In [13]:
preds = read_geojson(predictions_path)

In [14]:
preds.head()

,class,CLU_ID,geometry
0,Other,6884CB75-57C8-45BB-9882-545420C97AFF,"MULTIPOLYGON (((-80.92192 40.63885, -80.92126 ..."
1,Corn,1FAFE575-B9E7-4922-930F-5A6C4C5C5D2D,"MULTIPOLYGON (((-80.93056 40.69504, -80.92921 ..."
2,Soybeans,53764DAD-4477-489F-BD6E-9E5D4043FB12,"MULTIPOLYGON (((-80.89612 40.63810, -80.89616 ..."
3,Corn,b1eda639-8d2e-49f8-84be-e42cf4efbf82,"MULTIPOLYGON (((-80.93498 40.69577, -80.93520 ..."
4,Other,4DD49095-57D8-4075-AA42-34D6B53364CA,"MULTIPOLYGON (((-80.89639 40.64179, -80.89650 ..."


In [19]:
def create_style(class_):
    colors = dict(soybeans='#6F634B',
                  corn='#fbec5d')
    
    style = dict(color=colors.get(class_.lower(), '#C0C0C0'), 
                 popup_text=class_.lower(), 
                )    
    return str(style)

In [20]:
preds['style'] = preds['class'].apply(lambda cl: create_style(cl))

In [21]:
preds.head(3)

,class,CLU_ID,geometry,style
0,Other,6884CB75-57C8-45BB-9882-545420C97AFF,"MULTIPOLYGON (((-80.92192 40.63885, -80.92126 ...","{'color': '#C0C0C0', 'popup_text': 'other'}"
1,Corn,1FAFE575-B9E7-4922-930F-5A6C4C5C5D2D,"MULTIPOLYGON (((-80.93056 40.69504, -80.92921 ...","{'color': '#fbec5d', 'popup_text': 'corn'}"
2,Soybeans,53764DAD-4477-489F-BD6E-9E5D4043FB12,"MULTIPOLYGON (((-80.89612 40.63810, -80.89616 ...","{'color': '#6F634B', 'popup_text': 'soybeans'}"


In [26]:
print("Generating styled result .geojson file...")

stem = Path(predictions_path).stem
temp_path = os.path.join(BASE, f"{RESULTSDIR}/{stem}.geojson.temp") 
os.makedirs(os.path.dirname(temp_path), exist_ok=True)

preds.to_file(temp_path, driver='GeoJSON')
    
print(f"Generated temp file: {temp_path}")
output_path = temp_path[:-5]
print(f"Rename temp file: {temp_path} to {output_path}")
os.rename(temp_path, output_path)

print("Execution finished")

Generating styled result .geojson file...
Generated temp file: /home/jovyan/work/results/pw/clu_predictions_2020_reformatted.geojson.temp
Rename temp file: /home/jovyan/work/results/pw/clu_predictions_2020_reformatted.geojson.temp to /home/jovyan/work/results/pw/clu_predictions_2020_reformatted.geojson
Execution finished
